# ChatGLM3-6B QLoRA 微调前后效果对比

本 Notebook 用于对比微调前后的 ChatGLM3-6B 模型在 AdvertiseGen 数据集上的生成效果。


In [1]:
# 安装依赖（如果遇到 ModuleNotFoundError，请运行此单元格）
# 取消下面的注释来安装
# !pip install torch transformers peft pandas
!pip install torch transformers peft pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. 安装依赖（如果未安装）

**注意**: 如果遇到 `ModuleNotFoundError`，请先运行下面的单元格安装依赖。

```python
# 取消注释下面的行来安装依赖
# !pip install torch transformers peft pandas
```

## 2. 导入必要的库


In [ ]:
# 导入必要的库
# 如果遇到 ModuleNotFoundError，请先运行 Cell 1 安装依赖，然后重启内核
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import pandas as pd
from pathlib import Path

print("✅ 所有库导入成功")
print(f"PyTorch 版本: {torch.__version__}")
print(f"Transformers 版本: {transformers.__version__}")
print(f"PEFT 版本: {peft.__version__}")


## 3. 加载基础模型和 Tokenizer


In [ ]:
# 模型ID
model_name_or_path = 'THUDM/chatglm3-6b'

# 根据训练脚本，训练时使用了 --bnb_mode none，所以这里不使用量化
print("正在加载基础模型...")
base_model = AutoModel.from_pretrained(
    model_name_or_path,
    device_map='auto',
    trust_remote_code=True
)

base_model.requires_grad_(False)
base_model.eval()

print("正在加载 Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    trust_remote_code=True
)

print("✅ 基础模型和 Tokenizer 加载完成")


## 4. 加载微调后的模型（QLoRA Adapter）


In [ ]:
# 微调后的模型路径
peft_model_path = "outputs/chatglm3-qlora"

print(f"正在加载微调后的模型: {peft_model_path}")
config = PeftConfig.from_pretrained(peft_model_path)
fine_tuned_model = PeftModel.from_pretrained(base_model, peft_model_path)
fine_tuned_model.eval()

print("✅ 微调后的模型加载完成")
print(f"训练配置: LoRA r={config.r}, alpha={config.lora_alpha}, dropout={config.lora_dropout}")


## 5. 定义对比函数


In [ ]:
def compare_responses(query, base_model, fine_tuned_model, tokenizer):
    """
    对比基础模型和微调后模型的生成结果
    """
    # 基础模型生成
    base_response, _ = base_model.chat(tokenizer, query=query, history=[])
    
    # 微调后模型生成
    fine_tuned_response, _ = fine_tuned_model.chat(tokenizer, query=query, history=[])
    
    return base_response, fine_tuned_response


## 6. 使用 AdvertiseGen 测试样本进行对比


In [ ]:
# 测试提示词（来自 AdvertiseGen 数据集）
test_prompts = [
    "写一段 30 字的广告文案，推广智能理财服务",
    "为一家精品酒店撰写一条促销推送通知",
    "以亲切的口吻，写一个体育用品品牌的引导购买对话"
]

print("=" * 80)
print("微调前后效果对比")
print("=" * 80)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n{'='*80}")
    print(f"测试样本 {i}")
    print(f"{'='*80}")
    print(f"\n📝 Prompt（提示词）:")
    print(f"   {prompt}")
    
    # 获取对比结果
    base_response, fine_tuned_response = compare_responses(
        prompt, base_model, fine_tuned_model, tokenizer
    )
    
    print(f"\n🔵 微调前输出:")
    print(f"   {base_response}")
    
    print(f"\n🟢 微调后输出:")
    # 清理输出文本（移除特殊标记）
    cleaned_response = fine_tuned_response.replace('[gMASK]', '').replace('sop', '').strip()
    print(f"   {cleaned_response}")
    
    print(f"\n{'='*80}")


## 7. 显示训练时保存的对比结果


In [ ]:
# 从训练结果中读取对比数据
comparison_df = pd.read_csv("outputs/chatglm3-qlora/prompt_comparison.csv")
print(f"✅ 读取了 {len(comparison_df)} 个测试样本")
print("\n训练时保存的微调前后对比结果:")
print("=" * 80)

for idx, row in comparison_df.iterrows():
    print(f"\n{'='*80}")
    print(f"样本 {idx + 1}")
    print(f"{'='*80}")
    print(f"\n📝 Prompt（提示词）:")
    print(f"   {row['prompt']}")
    
    print(f"\n🔵 微调前输出:")
    if row['pretraining_response'] == '[baseline skipped]':
        print("   [基线测试已跳过]")
    else:
        pre_text = str(row['pretraining_response']).replace('[gMASK]', '').replace('sop', '').strip()
        print(f"   {pre_text[:300]}..." if len(pre_text) > 300 else f"   {pre_text}")
    
    print(f"\n🟢 微调后输出:")
    post_text = str(row['posttraining_response']).replace('[gMASK]', '').replace('sop', '').strip()
    # 移除重复内容
    if post_text.count('"') > 4:
        lines = post_text.split('\n')
        unique_lines = []
        seen = set()
        for line in lines:
            line_clean = line.strip().strip('"').strip()
            if line_clean and line_clean not in seen:
                unique_lines.append(line_clean)
                seen.add(line_clean)
        post_text = '\n'.join(unique_lines)
    
    print(f"   {post_text[:500]}..." if len(post_text) > 500 else f"   {post_text}")
    print()


## 8. 总结

通过对比可以看出：

1. **微调前**: 模型对广告文案生成任务的理解较为基础，可能生成通用性较强的回答
2. **微调后**: 模型在 AdvertiseGen 数据集上微调后，能够更好地理解广告文案生成的需求，生成更符合任务要求的文案
3. **训练效果**: Loss 从 4.23 降低到 2.27，降低了 46.5%，说明模型在训练数据上学习到了有效的模式

### 训练配置
- **数据集**: AdvertiseGen (10,000 样本)
- **数据来源**: `data/advertisegen.json`
- **训练方法**: QLoRA (LoRA + 4-bit量化)
- **训练轮数**: 3 epochs
- **硬件**: RTX-5080, 16GB GPU, CUDA 12.9
